In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
from tqdm import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels=[]
for dirname, _, filenames in os.walk('/kaggle/input/dogs-vs-cats/train/train'):
    for filename in filenames:
        labels.append(filename.split('.')[0])
train_labels=labels

In [ ]:
device='cuda' if torch.cuda.is_available else 'cpu'
device

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class ResNet50(nn.Module):
    def __init__(self, num_classes=1):
        super(ResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
       
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)
    
model=ResNet50(1).to(device)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
train_lab=lb.fit_transform(train_labels)

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch.distributed as dist

In [ ]:
for param in  model.parameters():
    param.requires_grad=False
    
for param in model.resnet.fc.parameters():
    param.requires_grad=True
    
model = nn.DataParallel(model)

In [ ]:
from skimage.io import imread,imshow
from skimage.transform import resize
num_epochs=3
for epoch in range(num_epochs):
    losses=[]

    for dirname, _, filenames in os.walk('/kaggle/input/dogs-vs-cats/train/train'):
        for filename,label in tqdm(zip(filenames,train_lab),total=len(train_lab)):
            img=imread(os.path.join(dirname,filename))
            img=resize(img,(3,224,224))

            img=torch.tensor(img,dtype=torch.float)
            img=img.cuda(non_blocking=True)
            img=img.reshape(1,3,224,224)
            label=torch.tensor(label,dtype=torch.float)
            label=label.view(-1,1)
            label=label.cuda(non_blocking=True)
           
            outputs = model(img)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            losses.append(loss)
    print('mean loss for epoch {} is {} '.format(epoch,np.array(losses).mean()))